In [2]:
import cassandra
from cassandra.cluster import Cluster

In [5]:
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [6]:
try:
    session.execute("""select * from music_library""")
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


# 1. create keyspace

In [8]:
# repliction is the number of times we want our data to be copied
# since we have one node, we chose one
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication = {'class': 'SimpleStrategy', 'replication_factor': 1}"""
        )
except Exception as e:
    print(e)

# 2. connect to our keyspace

In [9]:
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

# 3. Create table

In [11]:
query = "create table if not exists music_library"
query = query + "(year int, artist_name text, album_name text, primary key (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## 4. Check if Table was created

In [12]:
query = "select count(*) from music_library"
try:
    count = session.execute(query)
except Exception as e:
    print(e)
print(count.one())

Row(count=0)


## 5. Insert rows

In [14]:
data = [(1979, "The Beatles", "Let it Be"), (1965, "The Beatles", "Rubber Soul")]

In [15]:
query = "insert into music_library (year, artist_name, album_name)"
query = query + "values (%s, %s, %s)"
for row in data:
    try:
        count = session.execute(query, row)
    except Exception as e:
        print(e)

## 6. Query Keyspace

In [17]:
query = "select * from music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for r in rows:
    print(r.year, r.artist_name, r.album_name)

1965 The Beatles Rubber Soul
1979 The Beatles Let it Be


In [18]:
query = "select * from music_library where year=1979"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for r in rows:
    print(r.year, r.artist_name, r.album_name)

1979 The Beatles Let it Be


## 7. Close and shutdown

In [19]:
session.shutdown()
cluster.shutdown()